# Title of notebook

Brief 1-2 sentence description of notebook.

In [1]:
import random

In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
from sklearn.preprocessing import StandardScaler
from collections import OrderedDict
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [4]:
from sklearn.metrics import confusion_matrix

In [5]:
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score

# importing random forest classifier from assemble module 
from sklearn.ensemble import RandomForestClassifier 


In [7]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

In [8]:
from sklearn.pipeline import Pipeline

In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [10]:
from sklearn.decomposition import PCA



In [11]:
from sklearn.decomposition import FastICA
import numpy as np

from sklearn.decomposition import PCA

In [12]:
from sklearn.preprocessing import StandardScaler


## Inputs & Data

Explanation of each input and where it comes from.

In [13]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

In [14]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

In [15]:
BAND_TO_COLOR = {"theta": "#FFAF00", "beta": "blue", "gamma": "#15616F"}
["#FFAF00", "#15616F"]

['#FFAF00', '#15616F']

In [16]:
OUTCOME_TO_COLOR = {'lose': "#951a1d",
 'omission': "#af780d",
 'rewarded': "#0499af",
 'win': "#3853a3",
'lose_comp': "#951a1d",
 'win_comp': "#3853a3",
'lose_non_comp': "#e67073",
 'win_non_comp': "#93a5da"}

In [17]:
MEASURE_TO_PATTERN = {"granger": "o", "coherence": "x", "power": ""}

In [18]:
GROUPING = "trial_outcome"
# GROUPING = "competition_closeness"

In [19]:
OUTCOME_LABELS = ["win", "lose", "rewarded"]
# OUTCOME_LABELS = ['win_non_comp', 'lose_non_comp', 'lose_comp', 'win_comp', 'rewarded']
# OUTCOME_LABELS = ['win_non_comp', 'lose_non_comp', 'lose_comp', 'win_comp']

# OUTCOME_LABELS = ['lose_non_comp', 'win_comp']

In [20]:
granger_df = pd.read_pickle("./proc/per_trial_granger.pkl")

In [21]:
power_df = pd.read_pickle("./proc/rce_sleap_and_power.pkl")

In [22]:
coherence_df = pd.read_pickle("./proc/rce_sleap_and_coherence.pkl")

In [23]:
granger_df.shape

(278, 126)

In [24]:
coherence_df.shape

(275, 128)

In [25]:
power_df.shape

(278, 161)

In [26]:
ALL_BANDS = ["theta", "beta", "gamma"]
BAND_TO_FREQ = {"theta": {"low_freq": 6, "high_freq": 11}, "beta": {"low_freq": 20, "high_freq": 31}, "gamma": {"low_freq": 30, "high_freq": 51}}
BAND_TO_FREQ = {"theta": (6,11), "beta": (20,31), "gamma": (30,51)}


In [27]:
merge_columns = ["time", "recording_file", "current_subject", "video_number"]
# Find columns in df2 that are not in merge_columns and also exist in df1, then drop them from df2
cols_to_drop = [col for col in power_df.columns if col not in merge_columns and col in coherence_df.columns]
coherence_df = coherence_df.drop(columns=cols_to_drop)
cols_to_drop = [col for col in power_df.columns if col not in merge_columns and col in granger_df.columns]
granger_df = granger_df.drop(columns=cols_to_drop)


In [28]:
power_col = [col for col in power_df.columns if "power_window-averaged-velocity-parsed" in col]

In [29]:
power_df = power_df.dropna(subset=power_col).reset_index(drop=True)

In [30]:
for col in power_col:
    region_1 = col.split("_")[0]
    
    for band, (low_freq, high_freq) in BAND_TO_FREQ.items():
        region_band_column = "{}_{}_band_power".format(region_1, band)
        power_df[region_band_column] = power_df[col].apply(lambda x: np.mean(x[low_freq: high_freq]))

In [31]:
granger_col = [col for col in granger_df.columns if "window_mean_granger" in col]

In [32]:
BAND_TO_FREQ

{'theta': (6, 11), 'beta': (20, 31), 'gamma': (30, 51)}

In [33]:
for col in granger_col:
    region_1 = col.split("_")[0]
    region_2 = col.split("_")[1]
    
    for band, (low_freq, high_freq) in BAND_TO_FREQ.items():
        region_band_column = "{}_{}_{}_band_granger".format(region_1, region_2, band)
        granger_df[region_band_column] = granger_df[col].apply(lambda x: np.mean(x[low_freq: high_freq]))

/tmp/ipykernel_1814845/202851467.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  granger_df[region_band_column] = granger_df[col].apply(lambda x: np.mean(x[low_freq: high_freq]))
/tmp/ipykernel_1814845/202851467.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  granger_df[region_band_column] = granger_df[col].apply(lambda x: np.mean(x[low_freq: high_freq]))
/tmp/ipykernel_1814845/202851467.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has p

In [34]:
MERGED_LFP_AND_SLEAP = pd.merge(left=power_df, right=coherence_df, on=merge_columns, how="inner")

In [35]:
MERGED_LFP_AND_SLEAP = pd.merge(left=MERGED_LFP_AND_SLEAP, right=granger_df, on=merge_columns, how="inner")

In [36]:
MERGED_LFP_AND_SLEAP

,time,recording_file,video_number,current_subject,mPFC_baseline_lfp_trace,mPFC_trial_lfp_trace,mPFC_baseline-trial_lfp_trace,vHPC_baseline_lfp_trace,vHPC_trial_lfp_trace,vHPC_baseline-trial_lfp_trace,...,MD_vHPC_gamma_band_granger,vHPC_MD_theta_band_granger,vHPC_MD_beta_band_granger,vHPC_MD_gamma_band_granger,mPFC_vHPC_theta_band_granger,mPFC_vHPC_beta_band_granger,mPFC_vHPC_gamma_band_granger,vHPC_mPFC_theta_band_granger,vHPC_mPFC_beta_band_granger,vHPC_mPFC_gamma_band_granger
0,30950500,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,"[-0.3134648, -0.39993784, -0.8777014, -1.44193...","[0.50154364, 0.29617018, 0.22050627, 0.1145767...","[-0.3134648, -0.39993784, -0.8777014, -1.44193...","[0.2583401, -0.26769176, -0.8311303, -1.046219...","[-1.1479185, -1.1806493, -1.1432426, -1.139735...","[0.2583401, -0.26769176, -0.8311303, -1.046219...",...,0.308700,0.544461,0.315053,0.392972,0.241112,0.322984,0.305378,0.488346,0.492734,0.384876
1,32550496,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,"[1.1176641, 1.0657803, 0.9987637, 0.8128466, 0...","[0.31995025, 0.16429879, 0.010809131, 0.067016...","[1.1176641, 1.0657803, 0.9987637, 0.8128466, 0...","[1.8691666, 2.0795794, 2.2408957, 2.3461022, 2...","[-1.0205019, -0.9152954, -0.9947847, -0.976081...","[1.8691666, 2.0795794, 2.2408957, 2.3461022, 2...",...,0.296993,0.424742,0.464322,0.484576,0.337904,0.300037,0.328714,0.486609,0.406057,0.452749
2,34350494,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,"[-0.22482991, 0.3567013, 0.12538591, -0.136195...","[-0.13835686, 0.42804158, 0.5945022, 0.3134648...","[-0.22482991, 0.3567013, 0.12538591, -0.136195...","[-1.0017985, -0.70955855, -0.7002069, -0.68033...","[0.47109076, 0.34016728, 0.18352668, 0.174175,...","[-1.0017985, -0.70955855, -0.7002069, -0.68033...",...,0.264659,0.416857,0.538528,0.528551,0.379768,0.296710,0.271932,0.515236,0.571965,0.458858
3,35950496,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,"[0.4864109, 0.18591705, 0.397776, 0.24212453, ...","[-0.32211208, 0.023780087, 0.13835686, -0.1102...","[0.4864109, 0.18591705, 0.397776, 0.24212453, ...","[0.58447987, 0.4675839, 0.21275067, 0.03389983...","[0.15780956, 0.047927346, -0.14728892, -0.3565...","[0.58447987, 0.4675839, 0.21275067, 0.03389983...",...,0.336984,0.406630,0.523946,0.504038,0.235804,0.441383,0.310826,0.403164,0.538562,0.466377
4,37450494,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,"[-3.6210587, -2.9530544, -2.4342163, -2.574735...","[-2.9508927, -2.998453, -2.3282866, -2.0104983...","[-3.6210587, -2.9530544, -2.4342163, -2.574735...","[-1.4191171, -1.3793725, -1.1233703, -0.911788...","[-4.0457697, -4.2304654, -3.9113393, -3.448431...","[-1.4191171, -1.3793725, -1.1233703, -0.911788...",...,0.269036,0.535375,0.556873,0.593945,0.419996,0.330796,0.258950,0.467323,0.428740,0.466091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,65281838,20230621_111240_standard_comp_to_omission_D5_s...,1.0,1.4,"[0.25026315, -0.0152599495, 0.03967587, 0.0122...","[-2.2889924, -2.2004848, -1.4741111, -1.010208...","[0.25026315, -0.0152599495, 0.03967587, 0.0122...","[-0.54008055, -0.7600229, -0.74047244, -0.6133...","[-0.8479998, -1.0703859, -0.95063955, -0.75513...","[-0.54008055, -0.7600229, -0.74047244, -0.6133...",...,0.266266,0.299642,0.297781,0.298878,0.370794,0.256748,0.272788,0.367977,0.282358,0.261535
266,66781856,20230621_111240_standard_comp_to_omission_D5_s...,1.0,1.4,"[0.42422658, 0.38760272, 0.28383505, 0.0427278...","[-2.426332, -2.2187965, -1.6297625, -1.0193646...","[0.42422658, 0.38760272, 0.28383505, 0.0427278...","[-0.82844937, -0.7673543, -0.7062592, -0.71359...","[-2.074789, -1.8841724, -1.4002994, -1.0117346...","[-0.82844937, -0.7673543, -0.7062592, -0.71359...",...,0.339557,0.327391,0.343262,0.252074,0.395614,0.231199,0.343809,0.307409,0.337629,0.300374
267,68781880,20230621_111240_standard_comp_to_omission_D5_s...,1.0,1.4,"[0.64702183, 0.027467908, -0

In [37]:
MERGED_LFP_AND_SLEAP["recording_file"].unique()

array(['20221214_125409_om_and_comp_6_1_top_1_base_2_vs_6_3',
       '20221215_145401_comp_amd_om_6_1_top_4_base_3',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged',
       '20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged',
       '20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged'],
      dtype=object)

In [38]:
MERGED_LFP_AND_SLEAP.groupby(["time", "trial_outcome", "current_subject"]).count()

recording_file  video_number  \
time     trial_outcome current_subject                                 
1934930  lose          1.1                           1             1   
         win           1.4                           1             1   
4334936  lose          1.4                           1             1   
         win           1.1                           1             1   
4509412  lose          1.1                           1             1   
...                                                ...           ...   
68781880 rewarded      1.4                           1             1   
69560954 rewarded      1.1                           1             1   
                       1.2                           1             1   
70681909 omission      1.4                           1             1   
71681918 rewarded      1.4                           1             1   

                                        mPFC_baseline_lfp_trace  \
time     trial_outcome current_subject                            
1934930  lose          1.1                                    1   
         win           1.4                                    1   
4334936  lose          1.4                                    1   
         win           1.1                                    1   
4509412  lose          1.1                                    1   
...                                                         ...   
68781880 rewarded      1.4                                    1   
69560954 rewarded      1.1                                    1   
                       1.2                                    1   
70681909 omission      1.4                                    1   
71681918 rewarded      1.4                                    1   

                                        mPFC_trial_lfp_trace  \
time     trial_outcome current_subject                         
1934930  lose          1.1                                 1   
         win           1.4                                 1   
4334936  lose          1.4                                 1   
         win           1.1                                 1   
4509412  lose          1.1                                 1   
...                                                      ...   
68781880 rewarded      1.4                                 1   
69560954 rewarded      1.1                                 1   
                       1.2                                 1   
70681909 omission      1.4                                 1   
71681918 rewarded      1.4                                 1   

                                        mPFC_baseline-trial_lfp_trace  \
time     trial_outcome current_subject                                  
1934930  lose          1.1                                          1   
         win           1.4                                          1   
4334936  lose          1.4                                          1   
         win           1.1                                          1   
4509412  lose          1.1                                          1   
...                                                               ...   
68781880 rewarded      1.4                                          1   
69560954 rewarded      1.1                                          1   
                       1.2                                          1   
70681909 omission      1.4                                          1   
71681918 rewarded      1.4                                          1   

                                        vHPC_baseline_lfp_trace  \
time     trial_outcome current_subject                            
1934930  lose          1.1                                    1   
         win           1.4                                    1   
4334936  lose          1.4                                    1   
         win           1.1                                    1   
4509412  lose          1.1                    

In [39]:
MERGED_LFP_AND_SLEAP[MERGED_LFP_AND_SLEAP["trial_outcome"] == "lose"].shape

(79, 362)

In [40]:
MERGED_LFP_AND_SLEAP["recording_dir"].unique()

array(['20221214_125409_om_and_comp_6_1_and_6_3',
       '20221215_145401_comp_amd_om_6_1_and_6_3',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2'],
      dtype=object)

In [41]:
feature_columns = [col for col in MERGED_LFP_AND_SLEAP if "band" in col and "beta" not in col]

In [42]:
MERGED_LFP_AND_SLEAP = MERGED_LFP_AND_SLEAP.dropna(subset=feature_columns)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

In [43]:
def get_region_network(row, columns, low_index=0, high_index=1):
    directed_graph = nx.DiGraph()
    for col in columns:
        region_1 = col.split("_")[0]
        region_2 = col.split("_")[1]
        directed_graph.add_edge(region_1, region_2, weight=np.mean(row[col][low_index:high_index]))
    return directed_graph

In [44]:
def generate_pairs(lst):
    pairs = []
    n = len(lst)
    for i in range(n):
        for j in range(i+1, n):
            pairs.append((lst[i], lst[j]))
    return pairs

# Part 1. Combining all the dataframes

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [173]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [174]:
ALL_BANDS = ["theta", "beta", "gamma"]
# BAND_TO_FREQ = {"theta": {"low_freq": 4, "high_freq": 12}, "beta": {"low_freq": 13, "high_freq": 30}, "gamma": {"low_freq": 30, "high_freq": 70}}
# BAND_TO_FREQ = {"theta": {"low_freq": 6, "high_freq": 11}, "gamma": {"low_freq": 30, "high_freq": 51}}
BAND_TO_FREQ = {"theta": (6, 11), "gamma": (30, 51)}

- Filtering for powers that are from low velocity parts of the trial

In [175]:
MERGED_LFP_AND_SLEAP.head()

,time,recording_file,video_number,current_subject,recording_dir,time_stamp_index,video_file,video_frame,subject_info,competition_closeness,...,vHPC_MD_theta_band_granger,vHPC_MD_beta_band_granger,vHPC_MD_gamma_band_granger,mPFC_vHPC_theta_band_granger,mPFC_vHPC_beta_band_granger,mPFC_vHPC_gamma_band_granger,vHPC_mPFC_theta_band_granger,vHPC_mPFC_beta_band_granger,vHPC_mPFC_gamma_band_granger,group
0,30950500,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,20221214_125409_om_and_comp_6_1_and_6_3,27998597,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,28574,6_1_top_1_base_2_vs_6_3,win_non_comp,...,0.544461,0.315053,0.392972,0.241112,0.322984,0.305378,0.488346,0.492734,0.384876,win
1,32550496,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,20221214_125409_om_and_comp_6_1_and_6_3,29598593,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,30569,6_1_top_1_base_2_vs_6_3,win_non_comp,...,0.424742,0.464322,0.484576,0.337904,0.300037,0.328714,0.486609,0.406057,0.452749,win
2,34350494,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,20221214_125409_om_and_comp_6_1_and_6_3,31398591,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,32813,6_1_top_1_base_2_vs_6_3,win_non_comp,...,0.416857,0.538528,0.528551,0.379768,0.296710,0.271932,0.515236,0.571965,0.458858,win
3,35950496,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,20221214_125409_om_and_comp_6_1_and_6_3,32998593,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,34807,6_1_top_1_base_2_vs_6_3,win_non_comp,...,0.406630,0.523946,0.504038,0.235804,0.441383,0.310826,0.403164,0.538562,0.466377,win
4,37450494,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,20221214_125409_om_and_comp_6_1_and_6_3,34498591,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,36677,6_1_top_1_base_2_vs_6_3,win_non_comp,...,0.535375,0.556873,0.593945,0.419996,0.330796,0.258950,0.467323,0.428740,0.466091,win


In [176]:
for col in MERGED_LFP_AND_SLEAP.columns:
    print(col)

time
recording_file
video_number
current_subject
recording_dir
time_stamp_index
video_file
video_frame
subject_info
competition_closeness
video_name
all_subjects
trial_outcome
lfp_index
Cohort
spike_interface_mPFC
spike_interface_vHPC
spike_interface_BLA
spike_interface_LH
spike_interface_MD
file_path
start_frame
stop_frame
individual_subj
all_subj
recording_name
track_names
subject_and_tracks_list
subject_id
agent_id
corner_path
corner_parts
bottom_width
top_width
right_height
left_height
average_height
average_width
width_ratio
height_ratio
box_bottom_left
reward_port
box_top_left
box_top_right
box_bottom_right
mPFC_theta_band_power
mPFC_beta_band_power
mPFC_gamma_band_power
vHPC_theta_band_power
vHPC_beta_band_power
vHPC_gamma_band_power
BLA_theta_band_power
BLA_beta_band_power
BLA_gamma_band_power
LH_theta_band_power
LH_beta_band_power
LH_gamma_band_power
MD_theta_band_power
MD_beta_band_power
MD_gamma_band_power
BLA_LH_theta_band_coherence
BLA_LH_beta_band_coherence
BLA_LH_gamma_b

In [177]:
MERGED_LFP_AND_SLEAP = MERGED_LFP_AND_SLEAP.drop(columns=[col for col in MERGED_LFP_AND_SLEAP.columns if "trace" in col], errors="ignore")
MERGED_LFP_AND_SLEAP = MERGED_LFP_AND_SLEAP.drop(columns=[col for col in MERGED_LFP_AND_SLEAP.columns if "frequencies" in col], errors="ignore")
MERGED_LFP_AND_SLEAP = MERGED_LFP_AND_SLEAP.drop(columns=[col for col in MERGED_LFP_AND_SLEAP.columns if "window" in col], errors="ignore")
MERGED_LFP_AND_SLEAP = MERGED_LFP_AND_SLEAP.drop(columns=[col for col in MERGED_LFP_AND_SLEAP.columns if "velocity" in col], errors="ignore")
MERGED_LFP_AND_SLEAP = MERGED_LFP_AND_SLEAP.drop(columns=[col for col in MERGED_LFP_AND_SLEAP.columns if "locations" in col], errors="ignore")
MERGED_LFP_AND_SLEAP = MERGED_LFP_AND_SLEAP.drop(columns=[col for col in MERGED_LFP_AND_SLEAP.columns if "trial" in col and "trial_outcome" != col], errors="ignore")
MERGED_LFP_AND_SLEAP = MERGED_LFP_AND_SLEAP.drop(columns=[col for col in MERGED_LFP_AND_SLEAP.columns if "baseline" in col], errors="ignore")



In [178]:
MERGED_LFP_AND_SLEAP.to_pickle("./proc/rce_sleap_and_spectral.pkl")

In [179]:
MERGED_LFP_AND_SLEAP

,time,recording_file,video_number,current_subject,recording_dir,time_stamp_index,video_file,video_frame,subject_info,competition_closeness,...,vHPC_MD_theta_band_granger,vHPC_MD_beta_band_granger,vHPC_MD_gamma_band_granger,mPFC_vHPC_theta_band_granger,mPFC_vHPC_beta_band_granger,mPFC_vHPC_gamma_band_granger,vHPC_mPFC_theta_band_granger,vHPC_mPFC_beta_band_granger,vHPC_mPFC_gamma_band_granger,group
0,30950500,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,20221214_125409_om_and_comp_6_1_and_6_3,27998597,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,28574,6_1_top_1_base_2_vs_6_3,win_non_comp,...,0.544461,0.315053,0.392972,0.241112,0.322984,0.305378,0.488346,0.492734,0.384876,win
1,32550496,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,20221214_125409_om_and_comp_6_1_and_6_3,29598593,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,30569,6_1_top_1_base_2_vs_6_3,win_non_comp,...,0.424742,0.464322,0.484576,0.337904,0.300037,0.328714,0.486609,0.406057,0.452749,win
2,34350494,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,20221214_125409_om_and_comp_6_1_and_6_3,31398591,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,32813,6_1_top_1_base_2_vs_6_3,win_non_comp,...,0.416857,0.538528,0.528551,0.379768,0.296710,0.271932,0.515236,0.571965,0.458858,win
3,35950496,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,20221214_125409_om_and_comp_6_1_and_6_3,32998593,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,34807,6_1_top_1_base_2_vs_6_3,win_non_comp,...,0.406630,0.523946,0.504038,0.235804,0.441383,0.310826,0.403164,0.538562,0.466377,win
4,37450494,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,1.0,6.1,20221214_125409_om_and_comp_6_1_and_6_3,34498591,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,36677,6_1_top_1_base_2_vs_6_3,win_non_comp,...,0.535375,0.556873,0.593945,0.419996,0.330796,0.258950,0.467323,0.428740,0.466091,win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,64081824,20230621_111240_standard_comp_to_omission_D5_s...,1.0,1.4,20230621_111240_standard_comp_to_omission_D5_s...,60888780,20230621_111240_standard_comp_to_omission_D5_s...,60859,1-4_t3b3L_box1,rewarded,...,0.415546,0.385304,0.281132,0.325827,0.328302,0.312059,0.478434,0.322979,0.248852,rewarded
265,65281838,20230621_111240_standard_comp_to_omission_D5_s...,1.0,1.4,20230621_111240_standard_comp_to_omission_D5_s...,62088794,20230621_111240_standard_comp_to_omission_D5_s...,62056,1-4_t3b3L_box1,rewarded,...,0.299642,0.297781,0.298878,0.370794,0.256748,0.272788,0.367977,0.282358,0.261535,rewarded
266,66781856,20230621_111240_standard_comp_to_omission_D5_s...,1.0,1.4,20230621_111240_standard_comp_to_omission_D5_s...,63588812,20230621_111240_standard_comp_to_omission_D5_s...,63554,1-4_t3b3L_box1,rewarded,...,0.327391,0.343262,0.252074,0.395614,0.231199,0.343809,0.307409,0.337629,0.300374,rewarded
267,68781880,20230621_111240_standard_comp_to_omission_D5_s...,1.0,1.4,20230621_111240_standard_comp_to_omission_D5_s...,65588836,20230621_111240_standard_comp_to_omission_D5_s...,65550,1-4_t3b3L_box1,rewarded,...,0.522760,0.286215,0.240627,0.283623,0.252313,0.290051,0.421022,0.308356,0.282759,rewarded


- Filtering out for trials that are win vs loss vs rewarded

In [180]:
MERGED_LFP_AND_SLEAP = MERGED_LFP_AND_SLEAP[MERGED_LFP_AND_SLEAP["trial_outcome"] != "omission"]

# Using mixed model to see which features are different while taking into account individual subject differences

In [181]:
all_features = [col for col in MERGED_LFP_AND_SLEAP.columns if "theta" in col or "gamma" in col]

In [182]:
MERGED_LFP_AND_SLEAP["trial_outcome"]

0           win
1           win
2           win
3           win
4           win
         ...   
264    rewarded
265    rewarded
266    rewarded
267    rewarded
269    rewarded
Name: trial_outcome, Length: 252, dtype: object

In [185]:
from scipy.stats import zscore


In [191]:
MERGED_LFP_AND_SLEAP = MERGED_LFP_AND_SLEAP[~(MERGED_LFP_AND_SLEAP[all_features].apply(zscore) >= 3).any(axis=1)
]

In [252]:
# Get unique trial types
trial_types = MERGED_LFP_AND_SLEAP['trial_outcome'].unique()

# Store mdfs
results = []

# Iterating through each LFP features
for feature in all_features:
    
        
    # Fit the model with the new baseline
    md = smf.mixedlm(formula="{} ~ C(trial_outcome, Treatment('{}'))".format(feature, trial_type), data=MERGED_LFP_AND_SLEAP, groups="current_subject", re_formula = "~C(trial_outcome, Treatment('{}'))".format(trial_type))  

    free = sm.regression.mixed_linear_model.MixedLMParams.from_components(
    np.ones(3), np.eye(3)
)
    
    mdf = md.fit(free=free, method=["lbfgs"])
    # mdf = md.fit(method=["lbfgs"])

    # Extracting the summary as a DataFrame
    summary_df = pd.DataFrame({
        'Param': mdf.params.index,
        'Coef': mdf.params.values,
        'Std.Err': mdf.bse.values,
        'z': mdf.tvalues.values,
        'P>|z|': mdf.pvalues.values,
        'CI Lower': mdf.conf_int()[0],
        'CI Upper': mdf.conf_int()[1],
        'base_trial_type': trial_type,
        'feature': feature,
        
    })
    # Store the result
    results.append(summary_df)

/home/riwata/miniconda3/envs/spike_interface_0_99_0/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/riwata/miniconda3/envs/spike_interface_0_99_0/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/riwata/miniconda3/envs/spike_interface_0_99_0/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2207: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
/home/riwata/miniconda3/envs/spike_interface_0_99_0/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2219: ConvergenceWarning: Gradient optimization failed, |grad| = 2.368839
  warnings.warn(msg, Converg

In [253]:
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Mixed Linear Model Regression Results
==========================================================================================
Model:                  MixedLM      Dependent Variable:      vHPC_mPFC_gamma_band_granger
No. Observations:       187          Method:                  REML                        
No. Groups:             4            Scale:                   0.0010                      
Min. group size:        13           Log-Likelihood:          361.7699                    
Max. group size:        70           Converged:               Yes                         
Mean group size:        46.8                                                              
------------------------------------------------------------------------------------------
                                                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------------
Intercept                                        0.334    0.041  8.140 0.000  0.254  0.415
C(trial_outcome, Treatment('rewarded'))[T.lose]  0.008    0.006  1.489 0.137 -0.003  0.020
C(trial_outcome, Treatment('rewarded'))[T.win]  -0.005    0.006 -0.753 0.452 -0.017  0.008
Group Var                                        0.007    0.177                           
==========================================================================================

"""

In [254]:
print(mdf.summary())

                                                     Mixed Linear Model Regression Results
Model:                                   MixedLM                        Dependent Variable:                        vHPC_mPFC_gamma_band_granger
No. Observations:                        187                            Method:                                    REML                        
No. Groups:                              4                              Scale:                                     0.0009                      
Min. group size:                         13                             Log-Likelihood:                            363.5219                    
Max. group size:                         70                             Converged:                                 Yes                         
Mean group size:                         46.8                                                                                                  
---------------------------------------------

In [255]:
fig = plt.figure(figsize = (16, 9))

ax = sns.distplot(mdf.resid, hist = False, kde_kws = {"shade" : True, "lw": 1}, fit = stats.norm)

ax.set_title("KDE Plot of Model Residuals (Blue) and Normal Distribution (Black)")
ax.set_xlabel("Residuals")

NameError: name 'stats' is not defined

<Figure size 1600x900 with 0 Axes>

## Reformatting the results dataframe

- Combining all the dataframes

In [256]:
all_mixed_df  = pd.concat(results)

In [257]:
all_mixed_df = all_mixed_df.reset_index(drop=True)

- Removing all the unnecssary rows

In [258]:
all_mixed_df = all_mixed_df.dropna(subset="P>|z|")

all_mixed_df = all_mixed_df[~all_mixed_df["Param"].str.contains("Intercept")]

In [259]:
all_mixed_df = all_mixed_df[~all_mixed_df["Param"].str.contains("Cov")]
all_mixed_df = all_mixed_df[~all_mixed_df["Param"].str.contains("Var")]


In [260]:
all_mixed_df

,Param,Coef,Std.Err,z,P>|z|,CI Lower,CI Upper,base_trial_type,feature
1,"C(trial_outcome, Treatment('rewarded'))[T.lose]",0.004538,0.003947,1.149736,0.250253,-0.003198,0.012274,rewarded,mPFC_theta_band_power
2,"C(trial_outcome, Treatment('rewarded'))[T.win]",0.002794,0.002847,0.981210,0.326489,-0.002787,0.008375,rewarded,mPFC_theta_band_power
10,"C(trial_outcome, Treatment('rewarded'))[T.lose]",0.000261,0.000114,2.285135,0.022305,0.000037,0.000484,rewarded,mPFC_gamma_band_power
11,"C(trial_outcome, Treatment('rewarded'))[T.win]",0.000126,0.000104,1.210720,0.226003,-0.000078,0.000331,rewarded,mPFC_gamma_band_power
19,"C(trial_outcome, Treatment('rewarded'))[T.lose]",0.006107,0.005634,1.083934,0.278394,-0.004936,0.017150,rewarded,vHPC_theta_band_power
...,...,...,...,...,...,...,...,...,...
605,"C(trial_outcome, Treatment('rewarded'))[T.win]",0.002634,0.009567,0.275297,0.783088,-0.016118,0.021386,rewarded,mPFC_vHPC_gamma_band_granger
613,"C(trial_outcome, Treatment('rewarded'))[T.lose]",0.001826,0.013013,0.140308,0.888416,-0.023679,0.027330,rewarded,vHPC_mPFC_theta_band_granger
614,"C(trial_outcome, Treatment('rewarded'))[T.win]",0.032198,0.028506,1.129539,0.258670,-0.023672,0.088069,rewarded,vHPC_mPFC_theta_band_granger
622,"C(trial_outcome, Treatment('rewarded'))[T.lose]",0.006275,0.014360,0.436948,0.662149,-0.021871,0.034421,rewarded,vHPC_mPFC_gamma_band_granger


- Renaming the names of the trial outcome so that it's just the trial outcome name

In [261]:
all_mixed_df["Param"] = all_mixed_df["Param"].apply(lambda x: x.split(".")[-1].strip("]"))


- Getting the feature type (power, coherence, Granger)

In [262]:
all_mixed_df["feature_type"] = all_mixed_df["feature"].apply(lambda x: x.split("_")[-1])

- Putting both trials together so we can drop duplicates

In [263]:
all_mixed_df["both_trial_types"] = all_mixed_df.apply(lambda x: tuple(sorted([x["base_trial_type"], x["Param"]])), axis=1)

- Filtering for rows that have a significant p-value

In [264]:
def get_sig_stars(x, n_groups=3):
    if x <= 0.0001 / n_groups:
        return "****"
    elif x <= 0.001 / n_groups:
        return "***"
    elif x <= 0.01 / n_groups:
        return "**"
    elif x <= 0.05 / n_groups:
        return "*"
    return "*"

In [265]:
all_mixed_df["signficance"] = all_mixed_df["P>|z|"].apply(lambda x: get_sig_stars(x))



# [all_mixed_df["P>|z|"] <= 0.001 / 3].drop_duplicates(subset=["feature", "both_trial_types"]).sort_values(["feature_type", "feature", "both_trial_types"])

In [270]:
all_mixed_df[all_mixed_df["signficance"].isin(["**", "***", "****"])].drop_duplicates(subset=["feature", "both_trial_types"]).sort_values(["feature_type", "feature", "both_trial_types"])

,Param,Coef,Std.Err,z,P>|z|,CI Lower,CI Upper,base_trial_type,feature,feature_type,both_trial_types,signficance
100,lose,-0.019690,0.004368,-4.507754,6.551747e-06,-0.028251,-0.011129,rewarded,BLA_LH_gamma_band_coherence,coherence,"(lose, rewarded)",****
208,lose,-0.019967,0.004858,-4.109939,3.957632e-05,-0.029489,-0.010445,rewarded,LH_vHPC_gamma_band_coherence,coherence,"(lose, rewarded)",***
379,lose,0.037554,0.008882,4.228031,2.357456e-05,0.020145,0.054962,rewarded,BLA_vHPC_theta_band_granger,granger,"(lose, rewarded)",****
487,lose,0.044645,0.014330,3.115446,1.836669e-03,0.016558,0.072733,rewarded,LH_vHPC_theta_band_granger,granger,"(lose, rewarded)",**
541,lose,0.049574,0.016158,3.068079,2.154400e-03,0.017905,0.081243,rewarded,mPFC_MD_theta_band_granger,granger,"(lose, rewarded)",**
397,lose,0.081517,0.014692,5.548512,2.881110e-08,0.052722,0.110312,rewarded,vHPC_BLA_theta_band_granger,granger,"(lose, rewarded)",****
506,win,0.061377,0.018678,3.286049,1.016033e-03,0.024769,0.097985,rewarded,vHPC_LH_theta_band_granger,granger,"(rewarded, win)",**
28,lose,0.000724,0.000218,3.329763,8.692004e-04,0.000298,0.001151,rewarded,vHPC_gamma_band_power,power,"(lose, rewarded)",**


In [273]:
mdf.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                                     Mixed Linear Model Regression Results
===============================================================================================================================================
Model:                                   MixedLM                        Dependent Variable:                        vHPC_mPFC_gamma_band_granger
No. Observations:                        187                            Method:                                    REML                        
No. Groups:                              4                              Scale:                                     0.0009                      
Min. group size:                         13                             Log-Likelihood:                            363.5219                    
Max. group size:                         70                             Converged:                                 Yes                         
Mean group size:                         46.8                                                                                                  
-----------------------------------------------------------------------------------------------------------------------------------------------
                                                                                                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                             0.328    0.040  8.218 0.000  0.250  0.406
C(trial_outcome, Treatment('rewarded'))[T.lose]                                                       0.006    0.014  0.437 0.662 -0.022  0.034
C(trial_outcome, Treatment('rewarded'))[T.win]                                                       -0.003    0.020 -0.149 0.882 -0.042  0.036
current_subject Var                                                                                   0.005    0.208                           
current_subject x C(trial_outcome, Treatment('rewarded'))[T.lose] Cov                                 0.000    0.000                           
C(trial_outcome, Treatment('rewarded'))[T.lose] Var                                                   0.000    0.014                           
current_subject x C(trial_outcome, Treatment('rewarded'))[T.win] Cov                                  0.000    0.000                           
C(trial_outcome, Treatment('rewarded'))[T.lose] x C(trial_outcome, Treatment('rewarded'))[T.win] Cov  0.000    0.000                           
C(trial_outcome, Treatment('rewarded'))[T.win] Var                                                    0.000    0.056                           
===============================================================================================================================================

"""